# Data wrangling for crime data
1. Read data
2. Groupby time periods


In [10]:
import pandas as pd
import numpy as np
os.chdir('/Users/sarwaridas/Desktop/IDS 702/Final Project/ids720_CoffeeAndCrimes/')
crime = pd.read_csv(os.getcwd()+"/00_sourcedata/Crimes_-_2001_to_Present.csv")
crime=crime.loc[crime['Community Area']!=0]


In [11]:
crime = crime.drop(
    [
        "Updated On",
        "Latitude",
        "Longitude",
        "FBI Code",
        "X Coordinate",
        "Y Coordinate",
        "Case Number",
        "Block",
        "IUCR",
        "Arrest",
        "Domestic",
        "Beat",
    ],
    axis=1,
)
crime.nunique() #77 community areas

ID                      290058
Date                    269781
Primary Type                 2
Description                 31
Location Description       196
District                    23
Ward                        50
Community Area              77
Year                        21
Location                154683
dtype: int64

In [12]:
(100 * crime.isna().sum()) / len(crime)  # Missing 8.14% data for Wards

ID                      0.000000
Date                    0.000000
Primary Type            0.000000
Description             0.000000
Location Description    0.001379
District                0.001724
Ward                    8.147681
Community Area          8.153886
Year                    0.000000
Location                0.499211
dtype: float64

In [13]:
crime_by_year_CCA = crime.groupby(["Year", "Community Area"], as_index=False).count()
crime_by_year_CCA = crime_by_year_CCA.loc[crime_by_year_CCA["Year"] != 2001] #something is up with numbers for 2001
crime_by_year_CCA.sample(2) 

,Year,Community Area,ID,Date,Primary Type,Description,Location Description,District,Ward,Location
1432,2019,58.0,66,66,66,66,66,66,66,66
881,2012,46.0,291,291,291,291,291,291,291,291


In [14]:
import altair as alt
alt.Chart(crime_by_year_CCA).mark_circle(size=60).encode(
    x="Year:O", y="ID:Q", color="Ward:N", tooltip=["Ward", "Year", "ID"]
).interactive()


alt.Chart(...)

In [17]:
CCA = crime_by_year_CCA.Ward.unique()
input_dropdown = alt.binding_select(options=CCA)
selection = alt.selection_single(fields=["Community Area"], bind=input_dropdown, name="CCA:")
color = alt.condition(
    selection, alt.Color("Community Area:N", legend=None), alt.value("lightgray")
)

alt.Chart(crime_by_year_CCA).mark_point().encode(
    x=alt.X("Year:O", scale=alt.Scale(zero=False)),
    y=alt.Y("ID:Q", scale=alt.Scale(zero=False)),
    color="Community Area:N",
    tooltip=["Community Area", "Year", "ID"],
).add_selection(selection).transform_filter(selection)

alt.Chart(...)

In [18]:
os.chdir('/Users/sarwaridas/Desktop/IDS 702/Final Project/ids720_CoffeeAndCrimes/02_processed_data')
crime_by_year_CCA.to_csv("crime_by_year_CCA.csv")